In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D  
import jax
import jax.numpy as np
import pandas as pd
from scipy.interpolate import CloughTocher2DInterpolator
import seaborn as sns

from src.field import efield, hfield
from src.utils.dataloader import (load_antenna_el_properties,
                                  load_head_coords)
from src.utils.viz import fig_config, set_axes_equal

In [ ]:
print(f'platform: {jax.lib.xla_bridge.get_backend().platform}')

In [ ]:
jax.config.update("jax_enable_x64", True)

In [ ]:
sns.set(style='ticks', palette='colorblind')

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# frequency
f = 10e9

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
d = - 15 / 1000  # distance from the antenna

# human head
head_coords = load_head_coords()
target_area_coords = head_coords[
    (head_coords['y'] < 0) &
    (head_coords['x'] > 0) & (head_coords['x'] < target_area[0]) &
    (head_coords['z'] > 0) & (head_coords['z'] < target_area[1])]
target_area_coords.reset_index(drop=True, inplace=True)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# antenna position -- coordinates
xs = antenna_data.x.to_numpy() 
xs = xs + (target_area[0] - xs[-1]) / 2.
ys = np.zeros_like(xs) + target_area_coords['y'].min() + d
zs = np.zeros_like(xs)
zs = zs + (target_area[1] - zs[-1]) / 2.

In [ ]:
# visualize the exposure scenario

fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.plot(head_coords['x'], head_coords['y'], head_coords['z'], 'k.', alpha=0.02)
ax.plot(xs, ys, zs, c='r',
        label=(f'$\\lambda/2$ dipole antenna\n'
               f'$d = {-d * 1000}$ mm\n'
               f'$f = {f / 1e9}$ GHz'))
ax.set(xlabel='$x$ [m]', ylabel='$y$ [m]', zlabel='$z$ [m]',
       xticks=[-0.05, 0.05, 0.15],
       yticks=[0.0, 0.075, 0.15],
       zticks=[-0.05, 0.05, 0.15])
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.view_init(20, -40)
ax.legend()
ax = set_axes_equal(ax)
plt.tight_layout()
plt.show()

In [ ]:
# compute E field
E = target_area_coords.apply(
    lambda row: efield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
E.columns = ['Ex', 'Ey', 'Ez']
E_abs = E.apply(
    lambda row: np.sqrt(row['Ex'] ** 2 + row['Ey'] ** 2 + row['Ez'] ** 2),
    axis=1)
E.loc[:, 'E_abs'] = E_abs

# compute H field
H = target_area_coords.apply(
    lambda row: hfield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
H.columns = ['Hx', 'Hy', 'Hz']
H_abs = H.apply(
    lambda row: np.sqrt(row['Hx'] ** 2 + row['Hy'] ** 2 + row['Hz'] ** 2),
    axis=1)
H.loc[:, 'H_abs'] = H_abs

# update dataframe
target_area_calc = pd.concat([target_area_coords, E, H], axis=1)

# compute power density
def _pd(row):
    return (row['Ey'] * row['Hz'].conjugate()
            - row['Ez'] * row['Hy'].conjugate(),
            row['Ex'] * row['Hz'].conjugate(),
            row['Ex'] * row['Hy'].conjugate())

S = target_area_calc.apply(_pd, axis=1, result_type='expand')
S.columns = ['Sx', 'Sy', 'Sz']
S_abs = S.apply(
    lambda row: np.sqrt(row['Sx'] ** 2 + row['Sy'] ** 2 + row['Sz'] ** 2),
    axis=1)
S.loc[:, 'S_abs'] = S_abs 

# update dataframe
target_area_calc = pd.concat([target_area_calc, S], axis=1)

In [ ]:
# interpolate xz distribution of power density by using Clough-Tocher interpolation

x = target_area_calc['x'].to_numpy().reshape(-1, 1)
y = target_area_calc['y'].to_numpy().reshape(-1, 1)
z = target_area_calc['z'].to_numpy().reshape(-1, 1)
xz_plane_inp = np.hstack((x, z))
i = np.abs(target_area_calc['S_abs'].to_numpy(dtype=np.complex128))
xz_interp_fn = CloughTocher2DInterpolator(xz_plane_inp, i)
xz_grid = np.meshgrid(np.linspace(x.min(), x.max(), 101),
                      np.linspace(z.min(), z.max(), 101))
i_xz = xz_interp_fn(*xz_grid)

In [ ]:
# visualize the power density distribution over the target area of interest

fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = plt.axes(projection ='3d')
cs = ax.scatter(x, y, z, c=i, cmap='viridis')
cbar = fig.colorbar(cs, ax=ax, pad=0.1)
cbar.ax.set_ylabel(r'S [W/m$^2$]')

ax.plot(xs, ys, zs, c='r',
        label=(f'$\\lambda/2$ dipole antenna\n'
               f'$d = {-d * 1000}$ mm\n'
               f'$f = {f / 1e9}$ GHz'))
ax.set(xlabel='$x$ [m]', ylabel='$y$ [m]', zlabel='$z$ [m]',
       xticks=[0.0, 0.01, 0.02],
       yticks=[-0.04, -0.03, -0.02],
       zticks=[0.0, 0.01, 0.02])
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

ax.view_init(20, -40)
ax.legend()
fig.tight_layout();

In [ ]:
# visualize the interpolated distribution from the antenna point of view

fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(head_coords['x'], head_coords['z'], 'k.', alpha=0.05, zorder=-1)
ax.contourf(*xz_grid, i_xz, cmap='viridis')
ax.plot(xs, zs, 'r-')
ax.set(xlabel='$x$ [m]', ylabel='$z$ [m]',
       aspect='equal')
plt.show()

In [ ]:
# zoom in

fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.imshow(i_xz, origin='lower', cmap='viridis',
               extent=(x.min(), x.max(), z.min(), z.max()))
ax.plot(xs, zs, 'r-')
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel(r'S [W/m$^2$]')
ax.set(xlabel='$x$ [m]', ylabel='$z$ [m]',
       xticks=[0.005, 0.01, 0.015],
       yticks=[0.0025, 0.01, 0.0175])
fig.tight_layout();